# Projet MLOps Pompiers

In [52]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
import lightgbm as lgb

In [62]:
incidents = pd.read_csv(r"LFB_Incident_data_from_2009.csv")
mobilisation = pd.read_csv(r"LFB_Mobilisation_data_from_2009.csv")

C:\Users\w_sat\AppData\Local\Temp\ipykernel_3312\454423457.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  incidents = pd.read_csv(r"LFB_Incident_data_from_2009.csv")


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

## Exploration & Pre-processing

Dataset "Incidents"

In [3]:
incidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1592069 entries, 0 to 1592068
Data columns (total 39 columns):
 #   Column                                  Non-Null Count    Dtype  
---  ------                                  --------------    -----  
 0   IncidentNumber                          1592069 non-null  object 
 1   DateOfCall                              1592069 non-null  object 
 2   CalYear                                 1592069 non-null  int64  
 3   TimeOfCall                              1592069 non-null  object 
 4   HourOfCall                              1592069 non-null  int64  
 5   IncidentGroup                           1592069 non-null  object 
 6   StopCodeDescription                     1592069 non-null  object 
 7   SpecialServiceType                      506554 non-null   object 
 8   PropertyCategory                        1592069 non-null  object 
 9   PropertyType                            1592069 non-null  object 
 10  AddressQualifier              

In [4]:
incidents.isna().sum()

IncidentNumber                                  0
DateOfCall                                      0
CalYear                                         0
TimeOfCall                                      0
HourOfCall                                      0
IncidentGroup                                   0
StopCodeDescription                             0
SpecialServiceType                        1085515
PropertyCategory                                0
PropertyType                                    0
AddressQualifier                                0
Postcode_full                              778359
Postcode_district                               0
UPRN                                       141333
USRN                                       162871
IncGeo_BoroughCode                              0
IncGeo_BoroughName                              0
ProperCase                                      0
IncGeo_WardCode                                29
IncGeo_WardName                                29


In [5]:
incidents.duplicated().sum()

0

In [6]:
#Liste des colonnes à supprimer du dataframe incidents :
incid_cols = ['CalYear', 'HourOfCall', 'SpecialServiceType', 'AddressQualifier', 'Postcode_full',
             'UPRN', 'USRN','IncGeo_BoroughName', 'IncGeo_WardName', 'IncGeo_WardNameNew', 'ProperCase', 'Easting_m', 'Northing_m', 'Latitude',
               'Longitude', 'FRS','SecondPumpArriving_AttendanceTime',
                 'SecondPumpArriving_DeployedFromStation', 'Notional Cost (£)']

In [7]:
df_incid = incidents.drop(columns=incid_cols, axis=1)

In [8]:
df_incid.dropna(subset=['FirstPumpArriving_AttendanceTime', 'IncGeo_WardCode', 'IncidentStationGround', 'NumCalls'], inplace=True)

In [9]:
df_incid.isna().sum()

IncidentNumber                            0
DateOfCall                                0
TimeOfCall                                0
IncidentGroup                             0
StopCodeDescription                       0
PropertyCategory                          0
PropertyType                              0
Postcode_district                         0
IncGeo_BoroughCode                        0
IncGeo_WardCode                           0
Easting_rounded                           0
Northing_rounded                          0
IncidentStationGround                     0
FirstPumpArriving_AttendanceTime          0
FirstPumpArriving_DeployedFromStation    11
NumStationsWithPumpsAttending             0
NumPumpsAttending                         0
PumpCount                                 0
PumpMinutesRounded                        0
NumCalls                                  0
dtype: int64

In [10]:
# Remplacer les 11 NaN de DeployedFromStation par les valeurs respectives de IncidentStationGround :
df_incid['FirstPumpArriving_DeployedFromStation'] = df_incid['FirstPumpArriving_DeployedFromStation'].fillna(df_incid['IncidentStationGround'])
df_incid.isna().sum()

IncidentNumber                           0
DateOfCall                               0
TimeOfCall                               0
IncidentGroup                            0
StopCodeDescription                      0
PropertyCategory                         0
PropertyType                             0
Postcode_district                        0
IncGeo_BoroughCode                       0
IncGeo_WardCode                          0
Easting_rounded                          0
Northing_rounded                         0
IncidentStationGround                    0
FirstPumpArriving_AttendanceTime         0
FirstPumpArriving_DeployedFromStation    0
NumStationsWithPumpsAttending            0
NumPumpsAttending                        0
PumpCount                                0
PumpMinutesRounded                       0
NumCalls                                 0
dtype: int64

In [11]:
df_incid.shape

(1464222, 20)

In [12]:
df_incid['TimeOfCall'] = df_incid['DateOfCall'] + ' ' + df_incid['TimeOfCall']
df_incid.drop(['DateOfCall'], axis=1, inplace=True)
df_incid['TimeOfCall'] = pd.to_datetime(df_incid['TimeOfCall'], format='%d %b %Y %H:%M:%S')

In [13]:
df_incid['TimeOfCall'].head()

0   2009-01-01 00:00:37
2   2009-01-01 00:03:00
3   2009-01-01 00:04:27
4   2009-01-01 00:05:39
5   2009-01-01 00:06:03
Name: TimeOfCall, dtype: datetime64[ns]

In [14]:
df_incid.dtypes

IncidentNumber                                   object
TimeOfCall                               datetime64[ns]
IncidentGroup                                    object
StopCodeDescription                              object
PropertyCategory                                 object
PropertyType                                     object
Postcode_district                                object
IncGeo_BoroughCode                               object
IncGeo_WardCode                                  object
Easting_rounded                                   int64
Northing_rounded                                  int64
IncidentStationGround                            object
FirstPumpArriving_AttendanceTime                float64
FirstPumpArriving_DeployedFromStation            object
NumStationsWithPumpsAttending                   float64
NumPumpsAttending                               float64
PumpCount                                         int64
PumpMinutesRounded                              

## Dataset "Mobilisation"

In [15]:
mobilisation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2211937 entries, 0 to 2211936
Data columns (total 22 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   IncidentNumber            object 
 1   CalYear                   int64  
 2   HourOfCall                int64  
 3   ResourceMobilisationId    int64  
 4   Resource_Code             object 
 5   PerformanceReporting      object 
 6   DateAndTimeMobilised      object 
 7   DateAndTimeMobile         object 
 8   DateAndTimeArrived        object 
 9   TurnoutTimeSeconds        float64
 10  TravelTimeSeconds         float64
 11  AttendanceTimeSeconds     int64  
 12  DateAndTimeLeft           object 
 13  DateAndTimeReturned       object 
 14  DeployedFromStation_Code  object 
 15  DeployedFromStation_Name  object 
 16  DeployedFromLocation      object 
 17  PumpOrder                 int64  
 18  PlusCode_Code             object 
 19  PlusCode_Description      object 
 20  DelayCodeId             

In [16]:
mobilisation.isna().sum()

IncidentNumber                    0
CalYear                           0
HourOfCall                        0
ResourceMobilisationId            0
Resource_Code                     0
PerformanceReporting              0
DateAndTimeMobilised              0
DateAndTimeMobile             27161
DateAndTimeArrived                0
TurnoutTimeSeconds            27289
TravelTimeSeconds             27382
AttendanceTimeSeconds             0
DateAndTimeLeft               46356
DateAndTimeReturned         1238779
DeployedFromStation_Code         27
DeployedFromStation_Name         27
DeployedFromLocation           1098
PumpOrder                         0
PlusCode_Code                     0
PlusCode_Description              0
DelayCodeId                 1664927
DelayCode_Description       1664927
dtype: int64

In [17]:
mobilisation.duplicated().sum()

0

Conserver uniquement les 1ères forces arrivées sur site

In [18]:
# df_mobil = mobilisation[mobilisation['PumpOrder']==1]
df_mobil = mobilisation

In [19]:
df_mobil = df_mobil[['IncidentNumber', 'DateAndTimeMobilised','AttendanceTimeSeconds',
                     'DeployedFromStation_Name','DeployedFromLocation', 'PumpOrder', 'DelayCodeId']]

In [20]:
df_mobil['DelayCodeId'].value_counts()

DelayCodeId
12.0    324223
9.0     105593
5.0      34409
8.0      33632
7.0      18796
11.0      9404
13.0      8723
10.0      6764
6.0       2809
3.0       2657
Name: count, dtype: int64

In [21]:
# Remplacer les DelaiCodeId manquant par 1, supposant que le code 1 = pas de ralentissement.
df_mobil['DelayCodeId'] = df_mobil['DelayCodeId'].fillna(1)

C:\Users\w_sat\AppData\Local\Temp\ipykernel_3312\3068371128.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mobil['DelayCodeId'] = df_mobil['DelayCodeId'].fillna(1)


In [22]:
df_mobil.isna().sum()

IncidentNumber                 0
DateAndTimeMobilised           0
AttendanceTimeSeconds          0
DeployedFromStation_Name      27
DeployedFromLocation        1098
PumpOrder                      0
DelayCodeId                    0
dtype: int64

In [23]:
df_mobil = df_mobil.dropna()

In [24]:
df_mobil.dtypes

IncidentNumber               object
DateAndTimeMobilised         object
AttendanceTimeSeconds         int64
DeployedFromStation_Name     object
DeployedFromLocation         object
PumpOrder                     int64
DelayCodeId                 float64
dtype: object

In [25]:
df_mobil['DeployedFromLocation'].value_counts()

DeployedFromLocation
Home Station     2138496
Other Station      72336
Name: count, dtype: int64

In [26]:
df_mobil['DelayCodeId'] = df_mobil['DelayCodeId'].astype('int')

In [27]:
df_mobil.shape

(2210832, 7)

## Merge des deux Dataset

In [28]:
df = pd.merge(left=df_mobil, right=df_incid, on='IncidentNumber', how='left')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2210832 entries, 0 to 2210831
Data columns (total 25 columns):
 #   Column                                 Dtype         
---  ------                                 -----         
 0   IncidentNumber                         object        
 1   DateAndTimeMobilised                   object        
 2   AttendanceTimeSeconds                  int64         
 3   DeployedFromStation_Name               object        
 4   DeployedFromLocation                   object        
 5   PumpOrder                              int64         
 6   DelayCodeId                            int32         
 7   TimeOfCall                             datetime64[ns]
 8   IncidentGroup                          object        
 9   StopCodeDescription                    object        
 10  PropertyCategory                       object        
 11  PropertyType                           object        
 12  Postcode_district                      object        
 1

In [30]:
df['Year'] = df['TimeOfCall'].dt.year
df['DayOfWeek'] = df['TimeOfCall'].dt.dayofweek
df['Hour'] = df['TimeOfCall'].dt.hour

In [31]:
df.drop(columns=['IncidentNumber','TimeOfCall', 'DateAndTimeMobilised', 'DeployedFromStation_Name',
 'Postcode_district', 'PropertyType', 'NumCalls',
                  'FirstPumpArriving_AttendanceTime', 'FirstPumpArriving_DeployedFromStation'], axis = 1, inplace=True)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2210832 entries, 0 to 2210831
Data columns (total 19 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   AttendanceTimeSeconds          int64  
 1   DeployedFromLocation           object 
 2   PumpOrder                      int64  
 3   DelayCodeId                    int32  
 4   IncidentGroup                  object 
 5   StopCodeDescription            object 
 6   PropertyCategory               object 
 7   IncGeo_BoroughCode             object 
 8   IncGeo_WardCode                object 
 9   Easting_rounded                float64
 10  Northing_rounded               float64
 11  IncidentStationGround          object 
 12  NumStationsWithPumpsAttending  float64
 13  NumPumpsAttending              float64
 14  PumpCount                      float64
 15  PumpMinutesRounded             float64
 16  Year                           float64
 17  DayOfWeek                      float64
 18  Ho

In [33]:
df.head()

,AttendanceTimeSeconds,DeployedFromLocation,PumpOrder,DelayCodeId,IncidentGroup,StopCodeDescription,PropertyCategory,IncGeo_BoroughCode,IncGeo_WardCode,Easting_rounded,Northing_rounded,IncidentStationGround,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpMinutesRounded,Year,DayOfWeek,Hour
0,319,Home Station,1,1,Special Service,Special Service,Road Vehicle,E09000032,E05014010,528650.0,176850.0,Battersea,2.0,2.0,2.0,60.0,2009.0,3.0,0.0
1,342,Home Station,2,1,Special Service,Special Service,Road Vehicle,E09000032,E05014010,528650.0,176850.0,Battersea,2.0,2.0,2.0,60.0,2009.0,3.0,0.0
2,308,Home Station,1,1,Fire,Secondary Fire,Outdoor,E09000010,E05013682,533750.0,194450.0,Edmonton,1.0,1.0,1.0,60.0,2009.0,3.0,0.0
3,210,Home Station,1,1,Fire,Secondary Fire,Outdoor,E09000017,E05013571,507750.0,182850.0,Hillingdon,1.0,1.0,1.0,60.0,2009.0,3.0,0.0
4,233,Home Station,1,1,Fire,Secondary Fire,Outdoor,E09000019,E05013708,531050.0,185350.0,Holloway,1.0,2.0,2.0,60.0,2009.0,3.0,0.0


In [34]:
df.isna().sum()

AttendanceTimeSeconds                0
DeployedFromLocation                 0
PumpOrder                            0
DelayCodeId                          0
IncidentGroup                    26067
StopCodeDescription              26067
PropertyCategory                 26067
IncGeo_BoroughCode               26067
IncGeo_WardCode                  26067
Easting_rounded                  26067
Northing_rounded                 26067
IncidentStationGround            26067
NumStationsWithPumpsAttending    26067
NumPumpsAttending                26067
PumpCount                        26067
PumpMinutesRounded               26067
Year                             26067
DayOfWeek                        26067
Hour                             26067
dtype: int64

In [35]:
df = df.dropna()

In [36]:
# Profiling du dataframe incidents :
# profile = ProfileReport(df, title="Profiling Report")
# profile

## Get Dummies

In [37]:
dummies_cols = ['IncidentGroup','StopCodeDescription', 'PropertyCategory', 'DeployedFromLocation']

In [38]:
df = pd.get_dummies(df, columns=dummies_cols)

In [39]:
num_cols = ['NumStationsWithPumpsAttending', 'NumPumpsAttending', 'DelayCodeId']
df[num_cols] = df[num_cols].astype(int)

In [40]:
string_cols = ['IncGeo_BoroughCode', 'IncGeo_WardCode', 'IncidentStationGround']

df[string_cols] = df[string_cols].astype(str)

## Label Encoder

In [41]:
label_encoder = LabelEncoder()
df_encoded = df[string_cols].apply(label_encoder.fit_transform)

In [42]:
df.drop(string_cols, axis=1, inplace=True)

In [43]:
df = df.join(df_encoded)

In [44]:
df.dtypes

AttendanceTimeSeconds                                   int64
PumpOrder                                               int64
DelayCodeId                                             int32
Easting_rounded                                       float64
Northing_rounded                                      float64
NumStationsWithPumpsAttending                           int32
NumPumpsAttending                                       int32
PumpCount                                             float64
PumpMinutesRounded                                    float64
Year                                                  float64
DayOfWeek                                             float64
Hour                                                  float64
IncidentGroup_False Alarm                                bool
IncidentGroup_Fire                                       bool
IncidentGroup_Special Service                            bool
StopCodeDescription_AFA                                  bool
StopCode

## Correlation

In [45]:
correlation_matrix = df.corr()

In [46]:
correlation_matrix

,AttendanceTimeSeconds,PumpOrder,DelayCodeId,Easting_rounded,Northing_rounded,NumStationsWithPumpsAttending,NumPumpsAttending,PumpCount,PumpMinutesRounded,Year,...,PropertyCategory_Other Residential,PropertyCategory_Outdoor,PropertyCategory_Outdoor Structure,PropertyCategory_Rail Vehicle,PropertyCategory_Road Vehicle,DeployedFromLocation_Home Station,DeployedFromLocation_Other Station,IncGeo_BoroughCode,IncGeo_WardCode,IncidentStationGround
AttendanceTimeSeconds,1.000000,0.335209,0.564891,-0.042403,0.023270,0.198904,0.160018,0.107507,0.026135,0.001930,...,-0.028319,0.046038,-0.007984,-0.001059,0.038367,-0.053797,0.053797,-0.074726,0.012980,0.016478
PumpOrder,0.335209,1.000000,-0.161822,0.001830,0.004094,0.568412,0.639592,0.399550,0.044397,0.085803,...,0.055968,-0.084999,-0.073260,0.012040,-0.061557,-0.003550,0.003550,0.028394,-0.016122,-0.005878
DelayCodeId,0.564891,-0.161822,1.000000,-0.045840,0.012447,-0.098862,-0.158523,-0.094226,-0.002105,-0.020501,...,-0.046412,0.071614,0.020078,-0.007951,0.048675,-0.046828,0.046828,-0.084887,0.015603,0.022399
Easting_rounded,-0.042403,0.001830,-0.045840,1.000000,0.051488,0.020876,-0.002748,0.001257,0.004314,-0.000867,...,-0.019556,0.041387,0.037976,-0.003244,0.018975,-0.012757,0.012757,-0.008337,0.149808,-0.128093
Northing_rounded,0.023270,0.004094,0.012447,0.051488,1.000000,-0.013206,0.007284,0.006465,0.003098,-0.008775,...,-0.014014,-0.007235,0.009365,0.002809,0.007208,0.003360,-0.003360,-0.149106,-0.159202,0.005644
NumStationsWithPumpsAttending,0.198904,0.568412,-0.098862,0.020876,-0.013206,1.000000,0.881560,0.598074,0.137622,0.137674,...,0.065622,-0.103999,-0.089233,0.015590,-0.071889,-0.038226,0.038226,0.068582,-0.046177,-0.030784
NumPumpsAttending,0.160018,0.639592,-0.158523,-0.002748,0.007284,0.881560,1.000000,0.681901,0.162645,0.079226,...,0.083008,-0.122821,-0.106641,0.017361,-0.081844,-0.003356,0.003356,0.035207,-0.022788,-0.008502
PumpCount,0.107507,0.399550,-0.094226,0.001257,0.006465,0.598074,0.681901,1.000000,0.812166,0.058451,...,0.041900,-0.073319,-0.063963,0.010499,-0.054480,-0.002456,0.002456,0.016790,-0.011849,-0.004426
PumpMinutesRounded,0.026135,0.044397,-0.002105,0.004314,0.003098,0.137622,0.162645,0.812166,1.000000,0.008561,...,-0.011538,-0.003974,-0.002906,0.004777,-0.004056,-0.000772,0.000772,-0.005308,0.001638,0.000403
Year,0.001930,0.085803,-0.020501,-0.000867,-0.008775,0.137674,0.079226,0.058451,0.008561,1.000000,...,-0.049087,-0.029072,-0.035697,-0.002464,-0.021340,-0.023262,0.023262,-0.006042,0.008890,0.008814


Entrainement

In [47]:
X = df.drop('AttendanceTimeSeconds', axis=1)
y = df['AttendanceTimeSeconds']

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

Standardisation

In [53]:
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Linear Regression

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = lr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)
mpe = np.mean((y_test - y_pred) / y_test) * 100

# Affichage des métriques
print("Mean Squared Error (MSE): ", mse)
print("Mean Absolute Error (MAE): ", mae)
print("R-squared (R²): ", r2)
print("Root Mean Squared Error (RMSE): ", rmse)
print("Mean Percentage Error (MPE): ", mpe)

print("\nScore :")
print('score train :', lr.score(X_train, y_train))
print('score test :', lr.score(X_test, y_test))


Mean Squared Error (MSE):  11238.754340451564
Mean Absolute Error (MAE):  74.26034171581382
R-squared (R²):  0.5205644574006076
Root Mean Squared Error (RMSE):  106.01299137582886
Mean Percentage Error (MPE):  -36.359133232196164

Score :
score train : 0.5201283955828275
score test : 0.5205644574006076


LightGBM

In [55]:
import lightgbm as lgb

model = lgb.LGBMRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Calcul des métriques
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)
mpe = np.mean((y_test - y_pred) / y_test) * 100

# Affichage des métriques
print("Mean Squared Error (MSE): ", mse)
print("Mean Absolute Error (MAE): ", mae)
print("R-squared (R²): ", r2)
print("Root Mean Squared Error (RMSE): ", rmse)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057009 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1337
[LightGBM] [Info] Number of data points in the train set: 1747812, number of used features: 36
[LightGBM] [Info] Start training from score 350.236073
Mean Squared Error (MSE):  8635.156649261218
Mean Absolute Error (MAE):  65.94835698790067
R-squared (R²):  0.6319311527324711
Root Mean Squared Error (RMSE):  92.92554357797009


## Test prédictions

In [61]:
predictions = y_pred
y_true = y

# Générer un échantillon aléatoire pour l'affichage
indices = np.random.choice(len(predictions), size=5, replace=False)

# Afficher l'échantillon
for index in indices:
    print(f"Exemple {index + 1}:")
    print(f"Valeur prédite : {predictions[index]}")
    print(f"Valeur réelle : {y_true[index]}")
    print("-" * 20)

Exemple 308477:
Valeur prédite : 256.7301276019226
Valeur réelle : 583
--------------------
Exemple 113552:
Valeur prédite : 271.36450880460626
Valeur réelle : 249
--------------------
Exemple 249034:
Valeur prédite : 236.31518476484737
Valeur réelle : 266
--------------------
Exemple 98485:
Valeur prédite : 232.34147442824354
Valeur réelle : 328
--------------------
Exemple 386842:
Valeur prédite : 365.81494340962075
Valeur réelle : 468
--------------------
